# Gabriels Old Code

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import re

# Import ADS

In [ ]:
ADS_PATH = os.path.join('../resources/WECC_ADS/downloads/2032/Public Data')
ads= pd.read_csv(ADS_PATH + '/GeneratorList.csv',skiprows=2, encoding='unicode_escape')
ads = ads[ads['State'].isin(['NM', 'AZ', 'CA', 'WA', 'OR', 'ID', 'WY', 'MT', 'UT', 'SD', 'CO', 'NV', 'NE', '0', 'TX'])]
ads['Long Name'] = ads['Long Name'].astype(str)
ads['Commission Date'] = pd.to_datetime(ads['Commission Date'], format='#%Y-%m-%d#')
ads['Retirement Date'] = pd.to_datetime(ads['Retirement Date'], format='#%Y-%m-%d#')
ads['Name'] = ads['Name'].str.replace(" ", "")
ads['Name'] = ads['Name'].apply(lambda x: re.sub(r'[^a-zA-Z0-9]', '', x).lower())
ads['Long Name'] = ads['Long Name'].str.replace(" ", "")
ads['Long Name'] = ads['Long Name'].apply(lambda x: re.sub(r'[^a-zA-Z0-9]', '', x).lower())
ads['SubType'] = ads['SubType'].apply(lambda x: re.sub(r'[^a-zA-Z0-9]', '', x).lower())
ads.rename({'Name': 'ads_name', 'Long Name': 'ads_long_name', 'SubType': 'subtype','Commission Date':'commission_date','Retirement Date':'retirement_date','Area Name':'balancing_area'}, axis=1, inplace=True)
ads.rename(str.lower, axis='columns', inplace=True)
ads['long id'] = ads['long id'].astype(str)
ads.head(5)

In [ ]:
def split_long_id(row):
    if row['long id'].split('-')[0] == row['state']:
         id_list = row['long id'].split('-')[1:]
    else:
        id_list = row['long id']
    if len(id_list) > 2:
        vals=  pd.Series([id_list[0], id_list[1] + '-' + id_list[2]])
    elif len(id_list) == 2:
        vals= pd.Series([id_list[0], id_list[1]])
    elif len(id_list) == 1:
        vals= pd.Series([id_list[0], None])
    elif len(id_list) == 0:
        vals= pd.Series([None, None])
    vals.astype(str)
    return vals

# Apply the function to the 'long id' column
ads[['plant_id_ads','generator_id_ads']] = ads.apply(split_long_id, axis=1)
ads

In [ ]:
# ads['eia_id_ads'] = ads['long id'].str.split('-').str[1].fillna(99999)
# ads['generator_id_ads'] = ads['long id'].str.split('-').str[2].fillna(99999).astype(str)
# ads['eia_id_ads'] = ads['eia_id_ads'].replace('N/A', '9999').replace('??', '9999').replace(['',], '9999')
# ads.loc[~ads['eia_id_ads'].astype(str).str.isdigit(),['eia_id_ads']] = 99999  #replace all non integer values with 9999
# #TODO: THERE IS a wind plant (id: 57621) with three eia_ids that need to be split up since they exist.
# # ads.iloc[4864] #issue with this row
# # ads[~ads['eia_id_ads'].astype(str).str.isdigit()][['servicestatus', 'commission_date', 'retirement_date', 'devstatus','maxcap(mw)','eia_id_ads']].sort_values(by='maxcap(mw)', ascending=False).head(30)
# # ads.loc[ads[~ads['eia_id_ads'].astype(str).str.isdigit()]['maxcap(mw)'].sort_values(ascending=False).head(20).index]
# ads['eia_id_ads'] = ads['eia_id_ads'].astype(int).astype(str)
# # ads['gen_id_ads'] = ads['long id'].str.split('-').str[2].astype(str)
# ads.head(5)


## 2032 Network Preparation
### Populate ADS data with lat lon

In [ ]:
eia_ads_mapping = pd.read_csv('../repo_data/eia_mappings/eia_ads_generator_mapping.csv')
lat_dict = dict(zip(eia_ads_mapping['GeneratorKey'], eia_ads_mapping['latitude']))
lon_dict = dict(zip(eia_ads_mapping['GeneratorKey'], eia_ads_mapping['longitude']))
eia_id_dict = dict(zip(eia_ads_mapping['GeneratorKey'], eia_ads_mapping['plant_id_eia']))
# combine lat_dict and lon_dict into one dictionary
lat_lon_dict = {k: (lat_dict[k], lon_dict[k]) for k in lat_dict}
ads['lat'] = ads['generatorkey'].map(lat_lon_dict).str[0]
ads['lon'] = ads['generatorkey'].map(lat_lon_dict).str[1]
ads['plant_id_eia'] = ads['generatorkey'].map(eia_id_dict)
print('Total Num ADS Gens: ', ads.shape[0])
print('Number of ads with missing lat long: ', ads['lat'].isna().sum())
ads_matched = ads[~(ads['lat'].isnull() | ads['lon'].isnull())]
ads_matched
# bus_locs = ads.groupby('bus id').agg({'lat': 'mean', 'lon': 'mean'}).reset_index().dropna()
# ads[['lat', 'lon']] = ads[['lat', 'lon']].fillna(ads['bus id'].map(bus_locs.set_index('bus id')['lat']))
# ads[['lat', 'lon']] = ads[['lat', 'lon']].fillna(ads['bus id'].map(bus_locs.set_index('bus id')['lon']))
# print('Number of ads with missing lat long: ', ads['lat'].isna().sum())
# #didnt do anything

In [ ]:
#Trying other methods to match non-direct matches
ads_unmatched = ads[(ads['lat'].isnull() | ads['lon'].isnull())]
print(ads_unmatched.columns)
ads_unmatched = ads_unmatched[ads_unmatched.btm == '#FALSE#'] #TODO: filtering out BTM resources for now
eia_loc.plant_id_eia = eia_loc.plant_id_eia.astype(str)
matched_2 = pd.merge(ads_unmatched,eia_loc,left_on= 'plant_id_ads', right_on='plant_id_eia', how = 'inner')
matched_2 = matched_2.drop(columns=['lat','lon']).rename(columns={'Latitude':'lat','Longitude':'lon'})
ads_matched_2 = pd.concat([ads_matched, matched_2])
ads_unmatched = ads_unmatched[~ads_unmatched.generatorkey.isin(matched_2.generatorkey)]
# ads_unmatched.to_csv('ads_unmatched.csv')
ads_eia_mapping = ads_matched_2[['generatorkey','ads_name','long id', 'bus id', 'bus name', 'devstatus', 'balancing_area', 'region name', 'plant_id_ads','plant_id_eia', 'generator_id_ads', 'lat', 'lon']]
ads_eia_mapping.to_csv('/Users/kamrantehranchi/Local_Documents/pypsa-usa/workflow/repo_data/eia_mappings/eia_ads_generator_mapping_updated.csv',index=True)
ads_eia_mapping

## Historical Network Preparation
### Populate EIA data with thermal ramp rates

In [ ]:
#return datatype of each column in a eia_plant_locs
print(eia_plants_locs.dtypes)
eia_plants_locs.


In [ ]:
# eia_plants_locs.groupby('plant_id_eia').agg({'capacity_mw': 'mean', 'summer_capacity_mw': 'mean'})

In [ ]:
eia_plants_locs_thermal = eia_plants_locs[~eia_plants_locs.tech_type.isin(['Solar','Hydro','Wind','Storage',])]
eia_plants_locs_thermal

In [ ]:
pd.merge(eia_plants_locs_thermal, ads_eia_mapping, how='inner', left_on=['Plant Code','generator_id'], right_on=['plant_id_eia','generator_id_ads'])

In [ ]:
ads_eia_mapping.head(5)

In [ ]:
ads_thermal= pd.read_csv(ADS_PATH + '/Thermal_General_Info.csv',skiprows=1, )#encoding='unicode_escape')
ads_thermal.head(5)

In [ ]:
merged_thermal = pd.merge(eia_plants_locs, ads_eia_mapping[['plant_id_ads','generator_id_ads']], left_on=['plant_id_eia','generator_id'], right_on=['plant_id_ads','generator_id_ads'], how='left')
print(merged_thermal.columns)
merged_thermal